### Data Scraping
Para fazer o scraping dos dados do Google Play Store, iremos utilizar a biblioteca [google-play-scraper](https://github.com/JoMingyu/google-play-scraper).

Google-Play-Scraper fornece APIs para facilmente obter dados do Google Play Store utilizando Python sem dependências externas!

O primeiro passo é instalar a biblioteca:

In [ ]:
#Install Google play scraper: https://github.com/JoMingyu/google-play-scraper
!pip install google_play_scraper

Vamos importar as bibliotecas que iremos utilizar para fazer o download e tratamento dos dados de reviews do Google Play Store:

In [ ]:
import json
import pandas as pd
from tqdm import tqdm

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app

#### Aplicativos de Compra no Brasil

Referência: https://www.appannie.com/

Página: https://www.data.ai/pt/apps/google-play/top/downloads/brazil/shopping/android-all/?topchart=last-week

1. Shopee ⇒ com.shopee.br

2. Mercado Livre ⇒ com.mercadolibre

3. Magalu ⇒ com.luizalabs.mlapp

4. Americanas ⇒ com.b2w.americanas

5. Amazon Shopping ⇒ com.amazon.mShop.android.shopping

In [ ]:
apps_ids = ['com.shopee.br', 'com.mercadolibre', 'com.luizalabs.mlapp', 'com.b2w.americanas', 'com.amazon.mShop.android.shopping']

Vamos realizar o scraping dados para cada app. Nesta etapa iremos obter informações gerais dos apps.

In [ ]:
app_infos = []

for ap in tqdm(apps_ids):
    info = app(ap, lang='pt', country='br')
    del info['comments']
    app_infos.append(info)

In [ ]:
app_infos_df = pd.DataFrame(app_infos)
app_infos_df

#### Scraping reviews dos apps

O objetivo é criar um dataset balanceado com o mesmo número de reviews para cada score (1-5). A análise de sentimento será realizada utilizando o seguinte critério:

*   **Muito Negativo**: score 1
*   **Negativo**: score 2
*   **Neutro**: score 3
*   **Positivo**: score 4
*   **Muito Positivo**: score 5

Para termos um dataset balanceado entre avaliações negativas, neutras e positivas, iremos selecionar uma amostra de 10 avaliações para cada score e aplicativo.

Para cada app teremos 50 avaliações, sendo:
* 25 avaliações dos comentários mais relevantes;
* 25 avaliações dos comentários mais recentes;

In [ ]:
app_reviews = []

for ap in tqdm(apps_ids):
    for score in list(range(1, 6)):
        for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
            rvs, _ = reviews(
                ap,
                lang='pt',
                country='br',
                sort=sort_order,
                count= 5,
                filter_score_with=score
            )
            for r in rvs:
                r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
                r['appId'] = ap
            app_reviews.extend(rvs)

In [ ]:
len(app_reviews)

In [ ]:
app_reviews_df = pd.DataFrame(app_reviews)

## Distribuição dos reviews entre as classes de sentimentos
Vamos validar se a distribuição está balanceada entre os scores:

In [ ]:
app_reviews_df.score.value_counts().plot(kind = 'bar')

Agora iremos padronizar o dataset conforme previsto anteriormente:

*   **Muito Negativo**: score 1
*   **Negativo**: score 2
*   **Neutro**: score 3
*   **Positivo**: score 4
*   **Muito Positivo**: score 5

In [ ]:
class_names = ['Muito Negativo', 'Negativo', 'Neutro', 'Positivo', 'Muito Positivo']
def to_sentiment(rating):
  return class_names[int(rating) - 1]

# cria uma nova coluna 'sentiment' no dataframe
app_reviews_df['sentiment'] = app_reviews_df.score.apply(to_sentiment)

Depois de criada a nova coluna 'sentiment', iremos avaliar se o dataset está balanceado entre as classes negative, neutral e positive.

In [ ]:
app_reviews_df.sentiment.value_counts().plot(kind = 'bar')

## Salvando o Dataframe em CSV
**Armazenando** os reviews em um arquivo CSV.

In [ ]:
app_reviews_df.to_csv('reviews.csv', index=None, header=True)